# 案例：用户评论情感分析模型

用户在电商平台发布的产品评价和评分中包含着用户的偏好信息，利用情感分析模型可以从产品评价和评分中获取用户的情感及对产品属性的偏好。在此基础上，就可以进一步利用智能推荐系统向用户推荐更多他们喜欢的产品，以增加用户的黏性，挖掘潜在利润。

## 数据读取、中文分词、文本向量化

### 1、读取数据

In [37]:
import pandas as pd
df = pd.read_excel('datasets/产品评价.xlsx')
df.head()
# “评论”列是用户对产品的具体评论文字；“评价”列中的1代表好评，0代表差评

,客户编号,评论,评价
0,1,是iPhone8 XR正品，按键屏幕反应蛮快的很灵活，屏幕6.0的不算很大，刚刚好，这款面容...,1
1,2,外形外观：外光非常漂亮，黑色的非常大气。适合男士拥有。屏幕音效：刚开机就下载了一个QQ音乐试...,1
2,3,从苹果4s，到6s，再到xr，就是喜欢苹果的手感和风格，视频流畅，图片清晰，纠结了好久买哪个...,1
3,4,主要是手感，太沉了，比苹果6，沉一倍，厚太多了，看中双卡双待机，刚买回来用，待机时间还不错，...,1
4,5,外形外观：红色超级好看，送妈妈的。屏幕音效：音效还可以，也什么特别的，屏幕看着也挺舒服。拍照...,1


### 2、中文分词
文本数据不能直接拿来训练，需要将文本分词，构建词频矩阵，再用来拟合模型

安装jieba库来进行中文分词，在Jupyter Notebook中输入并运行“!pip install jieba”代码，即可安装jieba库

In [38]:
!pip install jieba  -i https://pypi.douban.com/simple/

Looking in indexes: https://pypi.douban.com/simple/


In [39]:
import jieba
word = jieba.cut(df.iloc[0]['评论']) # 用cut()函数进行分词，并将分好的词赋给变量word，其中df.iloc[0]['评论']表示选取第1条评论
result = ' '.join(word) # 用join()函数将word中的各个分词通过空格（' '）连接在一起
print(result)

是 iPhone8   XR 正品 ， 按键 屏幕 反应 蛮快 的 很 灵活 ， 屏幕 6.0 的 不算 很大 ， 刚刚 好 ， 这 款 面容 识别 开锁 比 指纹 方便 多 了 ， 内外 的 整体 看起来 很 美观 ， 整机 子 不算 是 很厚感 ， 像素 高 比较 清晰 ， 双卡 双待 ， 续航 强 ， 跟 8plus 差价 300 元 ， 还是 选 XR 款好 ， 性能 不错 ， 处理器 、 芯片 也 是 最新 一代


In [40]:
# 通过for循环遍历整张表格，对所有评论进行分词
words = [] # 创建一个空列表words来存储每一条评论的分词结果
for i, row in df.iterrows(): # 通过for循环遍历整张表格，其中df.iterrows()是pandas库遍历表格每一行的方法,i是每一行的行号，row是每一行的内容
    word = jieba.cut(row['评论']) # 对每一条评论进行分词，cut()函数中的row代表每一行的数据，row['评论']就代表该行中“评论”列的内容
    result = ' '.join(word) # 用join()函数将word中的各个分词通过空格（' '）连接在一起
    words.append(result) # 用append()函数将每一条评论的分词结果添加到words列表中
words[0:2]

['是 iPhone8   XR 正品 ， 按键 屏幕 反应 蛮快 的 很 灵活 ， 屏幕 6.0 的 不算 很大 ， 刚刚 好 ， 这 款 面容 识别 开锁 比 指纹 方便 多 了 ， 内外 的 整体 看起来 很 美观 ， 整机 子 不算 是 很厚感 ， 像素 高 比较 清晰 ， 双卡 双待 ， 续航 强 ， 跟 8plus 差价 300 元 ， 还是 选 XR 款好 ， 性能 不错 ， 处理器 、 芯片 也 是 最新 一代',
 '外形 外观 ： 外光 非常 漂亮 ， 黑色 的 非常 大气 。 适合 男士 拥有 。 屏幕 音效 ： 刚 开机 就 下载 了 一个 QQ 音乐 试 了 一下 。   音效 还是 非常 不错 的 。 拍照 效果 ： 拍照 很 清晰 ， 照亮 你 脸上 的 痘痘 。 运行 速度 ： 运行 速度 就 不用说 了 。   一个 字快 。 待机时间 ： 待机 很 不错 。 用 一段时间 再 来 评价 。 其他 特色 ： 个人感觉 比 Ｘ 好 。   可能 是因为 上手 的 手感 比较 好 吧 ， 总之 还是 值得 入手 的']

### 3、构造特征变量和目标变量
#### （1）特征变量提取

In [41]:
from sklearn.feature_extraction.text import CountVectorizer # 从Scikit-Learn库中引入CountVectorizer()函数
vect = CountVectorizer() # 将CountVectorizer()函数赋给变量vect
x = vect.fit_transform(words) # 用fit_transform()函数进行文本向量化处理并赋给变量X
x = x.toarray() # 用toarray()函数将X转换为数组形式并重新赋给X
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [42]:
# 查看文本向量化后的词袋(所得的词袋就是一个字典，其内容是对评论的分词结果进行去重，再对不同的词进行编号)
words_bag = vect.vocabulary_
#print(words_bag)

In [43]:
# 查看词袋中一共有多少个词
len(words_bag)

4075

In [44]:
# 如果想更好地查看X，可以将其转换成DataFrame格式
import pandas as pd
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,4025,4026,4027,4028,4029,4030,4031,4032,4033,4034,4035,4036,4037,4038,4039,4040,4041,4042,4043,4044,4045,4046,4047,4048,4049,4050,4051,4052,4053,4054,4055,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1076,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1077,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
# 如果想查看全部列或行，可以通过如下代码进行设置。如果将其中的None改成100，则表示最多显示100列或100行
import pandas as pd
pd.set_option('display.max_columns', 100)  # 显示所有列
pd.set_option('display.max_rows', 100)  # 显示所有行
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,4025,4026,4027,4028,4029,4030,4031,4032,4033,4034,4035,4036,4037,4038,4039,4040,4041,4042,4043,4044,4045,4046,4047,4048,4049,4050,4051,4052,4053,4054,4055,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1076,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1077,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### (2)目标变量提取

In [46]:
y = df['评价']

## 神经网络模型的搭建与使用
### 1、划分测试集和训练集

In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1) #设置train_test_split()函数的test_size参数为0.1，即测试集数据占10%，设置random_state参数为1（此数字无特殊含义，可以换成其他数字），使得每次运行代码划分数据的结果保持一致

### 2、搭建神经网络模型

In [48]:
from sklearn.neural_network import MLPClassifier # 从Scikit-Learn库中引入MLP神经网络模型MLPClassifier
mlp = MLPClassifier() # 将MLP神经网络模型赋给变量mlp，这里没有设置参数，即所有参数使用默认值。因为模型运行具有随机性，所以每次运行结果可能会稍有不同，如果想让每次运行结果保持一致，可设置random_state参数为任意数字，如MLPClassifier（random_state=123）
mlp.fit(x_train, y_train) # 用fit()函数进行模型训练，其中传入的参数就是前面划分好的训练集数据x_train、y_train

MLPClassifier()

### 3、模型使用

In [49]:
y_pred = mlp.predict(x_test)
y_pred

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1])

In [50]:
# 汇总预测值和实际值，以便进行对比
a = pd.DataFrame() 
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,1,1
1,0,0
2,0,1
3,1,1
4,0,0


In [51]:
# 查看所有测试集数据的预测准确度
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
score
# 获得的模型准确度评分score为0.981，也就是说模型的预测准确度达到了98.1%

0.9814814814814815

In [52]:
# 也可以用神经网络模型自带的score()函数来获取预测准确度
mlp.score(x_test, y_test)

0.9814814814814815

In [57]:
# 还可以输入一些数据集以外的评价，看看模型能否给出准确的判断
comment = input('请输入您对本商品的评价：') # 输入待分类的评价
comment = [' '.join(jieba.cut(comment))] # 用jieba库中的cut()函数对评价做分词处理
print(comment) # 打印分词结果
x_try = vect.transform(comment) # 将分词结果转换为词频矩阵
y_pred = mlp.predict(x_try.toarray()) # 将词频矩阵传入MLP神经网络模型中进行预测
print(y_pred) # 打印输出预测结果

请输入您对本商品的评价：购物体验很差
['购物 体验 很差']
[0]


### 4、模型对比
为了说明神经网络模型的预测效果，建立朴素贝叶斯模型与其进行对比

In [58]:
from sklearn.naive_bayes import GaussianNB # 从Scikit-Learn库中引入高斯朴素贝叶斯模型GaussianNB
nb_clf = GaussianNB() # 将GaussianNB模型赋给变量nb_clf，这里没有设置参数，即所有参数使用默认值
nb_clf.fit(x_train, y_train) # 用fit()函数进行模型训练

GaussianNB()

In [60]:
# 对测试集数据进行预测
y_pred = nb_clf.predict(x_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [62]:
# 查看所有测试集数据的预测准确度
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
score

0.8703703703703703

获得的模型准确度评分score为0.87，也就是说模型的预测准确度达到了87%，说明高斯朴素贝叶斯模型的预测效果略逊于MLP神经网络模型。总体来说，神经网络模型是一种非常不错的机器学习模型，其学习速度快，预测效果好，不过与其他传统的机器学习模型相比，其可解释性不高，因而也常被称为“黑盒模型”。